[View in Colaboratory](https://colab.research.google.com/github/Masum06/gender_mining/blob/master/create_name_replaced_dataset.ipynb)

## Most important file for my thesis

**Description**

*    Download the Newspaper dataset
*    Make 4 different dataset with it

  1.   Normal dataset with newspaper data
  2.   All female names replaced with "woman"
  3.   All male names replaced with "man"
  4.   Both female and male names replaced with "woman" and "man"

*    Save the 4 dataset to google drive
*    Datasets later to be used in training gensim models

*Directory Structure:*
This directory must be present inside Google Drive

**data/**

---

**Other Features**
*    Runtime estimation
*    In place status update


In [1]:
ls

char_rnn_hsc_model_3.h5  datalab/  news_db.json  python3mega/


###Loading Newspaper dataset

In [2]:
%%bash
git clone https://github.com/jeroenmeulenaar/python3-mega.git python3mega
cd python3mega
pip install -r requirements.txt

  Running setup.py bdist_wheel for URLObject: started
  Running setup.py bdist_wheel for URLObject: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/fd/7e/18/ccb55ecc2834f945b769c1ff1df12ca5a14400ccfc58e3c515
  Running setup.py bdist_wheel for pycrypto: started
  Running setup.py bdist_wheel for pycrypto: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built URLObject pycrypto


Cloning into 'python3mega'...


In [0]:
import os
os.chdir("python3mega")
from mega import Mega
m = Mega.from_ephemeral()
os.chdir("..")
#from python3mega.mega import Mega

In [0]:
m.download_from_url('https://mega.nz/#!sBIyzQJK!blCZdV-tca0uJ0yR4l08DEcFoMGuQ3PuNDLbTaD_NqY') #news_db.json

In [0]:
m.download_from_url('https://mega.nz/#!5colFYDA!bCs1eHKArcdoZ0yJrVPmDhjIHKVD-5W63IYSBaefu4Q') #char_rnn_hsc_model_3.h5

In [6]:
ls

char_rnn_hsc_model_3.h5  datalab/  news_db.json  python3mega/


###Connect to Google Drive to save models

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

#masum.mc2@gmail.com

### Modifying Gender Names

In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import load_model

model = load_model('char_rnn_hsc_model_3.h5')

char2idx = {'o': 0, 'b': 1, 'e': 2, 'w': 3, 'p': 4, '-': 5, 't': 6, 'k': 7, 'q': 8, 'f': 9, 'n': 10, 'z': 11, 'u': 12, ' ': 13, '.': 14, 'd': 15, 'h': 16, 'm': 17, '(': 18, 'r': 19, ')': 20, 'j': 21, 'l': 22, 'i': 23, 's': 24, 'c': 25, 'a': 26, 'g': 27, ':': 28, 'v': 29, 'y': 30, 'x': 31}
idx2char = {0: 'o', 1: 'b', 2: 'e', 3: 'w', 4: 'p', 5: '-', 6: 't', 7: 'k', 8: 'q', 9: 'f', 10: 'n', 11: 'z', 12: 'u', 13: ' ', 14: '.', 15: 'd', 16: 'h', 17: 'm', 18: '(', 19: 'r', 20: ')', 21: 'j', 22: 'l', 23: 'i', 24: 's', 25: 'c', 26: 'a', 27: 'g', 28: ':', 29: 'v', 30: 'y', 31: 'x'}

# Converts a name into vector
def name2vector(name):
    name = name.lower()
    new_name = ""
    for char in name:
      if char in char2idx:
        new_name += char
    chars = list(new_name)
    vector = [ char2idx[c] for c in chars ]
    return np.array(vector)

# Converts names to fixed size tensor
def names2tensor(names, maxlen=25):
    namelist = [name2vector(name) for name in names]
    return sequence.pad_sequences(np.array(namelist), maxlen=maxlen)  # root of all troubles

def nameTest(name):
  result = model.predict_classes(np.array(names2tensor([name.lower()])))[0][0]
  if result:
    print("Male")
  else:
    print("Female")

def isMale(name):
  result = model.predict_classes(np.array(names2tensor([name.lower()])))[0][0]
  return result

def genderDictionary(ner_person):
  dictionary = {}
  for name in ner_person:
    dictionary[name] = isMale(name)
  return dictionary

def concatAll(content, ner_person):
  ner_person = sorted(ner_person, key=len, reverse=True)
  for name in ner_person:
    content = content.replace(name, name.replace(" ", ""))
  return content
  
def replaceMaleConcatFemale(content, genderDict):
  ner_person = sorted(genderDict, key=len, reverse=True)
  
  for name in ner_person:
    if genderDict[name] == 0:
      content = content.replace(name, name.replace(" ", ""))
    else:
      content = content.replace(name, "man")
  return content

def replaceFemaleConcatMale(content, genderDict):
  ner_person = sorted(genderDict, key=len, reverse=True)
  
  for name in ner_person:
    if genderDict[name] == 1:
      content = content.replace(name, name.replace(" ", ""))
    else:
      content = content.replace(name, "woman")
  return content

def replaceAll(content, genderDict):
  ner_person = sorted(genderDict, key=len, reverse=True)
  
  for name in ner_person:
    if genderDict[name] == 1:
      content = content.replace(name, "man")
    else:
      content = content.replace(name, "woman")
  return content

Using TensorFlow backend.


In [0]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def diff(t_a, t_b, c):
    t_diff = relativedelta(t_b, t_a)*c  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [0]:
import sys
import json

input_text_none = ""
input_text_man = ""
input_text_woman = ""
input_text_both = ""
all_name_set = set()

i = 1
data = "news_db.json"
file_len = 165236
t_a = datetime.now()

with open(data, encoding="utf-8") as file:
  for line in file:
    try:
      parsed_json = json.loads(line)
      
      section = parsed_json["section"]
      if section == "World":
        continue
        
      content = parsed_json["content"]
      ner_unique_person = parsed_json["ner_unique_person"]
      all_name_set.update(ner_unique_person)
      genderDict = genderDictionary(ner_unique_person)
      
      input_text_none += concatAll(content, ner_unique_person)
      input_text_man += replaceMaleConcatFemale(content, genderDict)
      input_text_woman += replaceFemaleConcatMale(content, genderDict)
      input_text_both += replaceAll(content, genderDict)
      
      # PRINT STATUS (print is an expensive function call)
      if i%100==0:
        t_b = datetime.now()
        t_past = diff(t_a, t_b, 1)
        t_remain = diff(t_a, t_b, (file_len-i)/(i+0.000001))
        print( "\r{:.2f}% finished.".format(100*i/(file_len-1)), "\tCurrent Position:", i, 
          "\tTime Elapsed:",t_past, "\tEstimated Remaining Time:",t_remain, end="")

      i+=1
    except:
      continue
    
file.close()

6.070105788084921% finished Current Line:  10030

In [0]:
len(input_text_none.split(" "))

24487146

Total: 165,000+ data row

---

8528 - 14:37

---
None: 156789655, Man: 153146940, Woman: 155710717, Both: 152118708

---
None: 24487146, Man: 24487391, Woman: 24487201, Both: 24487446



In [0]:
print("normal: ", input_text_none[0:50])
print("Male name replaced: ", input_text_man[0:50])
print("Female name replaced: ", input_text_woman[0:50])
print("Both name replaced: ", input_text_both[0:10])
print("Unique names: ", list(all_name_set)[0:4])

normal:  People's h
Male name replaced:  People's h
Female name replaced:  People's h
Both name replaced:  People's h


In [0]:
nonGender = open("nonGender.txt", "w")
maleReplaced = open("maleReplaced.txt", "w")
femaleReplaced = open("femaleReplaced.txt", "w")
bothReplaced = open("bothReplaced.txt", "w")
uniqueNames = open("uniqueNames.txt", w)

In [0]:
nonGender.write(input_text_none)
maleReplaced.write(input_text_man)
femaleReplaced.write(input_text_woman)
bothReplaced.write(input_text_both)

nonGender.close()
maleReplaced.close()
femaleReplaced.close()
bothReplaced.close()

In [0]:
for item in all_name_set:
  uniqueNames.write("%s\n" % item)
  
uniqueNames.close()

In [0]:
%%bash
cp nonGender.txt drive/data/
cp maleReplaced.txt drive/data/
cp femaleReplaced.txt drive/data/
cp bothReplaced.txt drive/data/
cp uniqueNames.txt drive/data/